# Convert ROOT files to Trifeature HDF5 (slow)

The ROOT-digi files contain the calibrated events in NeuLAND. Here we extract the three basic properties number of hits, number of clusters, and total deposited energy and save them together with the number of incoming neutrons, number of primary points, and number of primary hits to a HDF5 file for basic reconstruction.

In [1]:
distances = [15]
doubleplanes = [12, 30]
energies = [600]
erels = [500]
neutrons = [1, 2, 3, 4, 5, 6]
physicss = ["inclxx"]
subruns = range(20)

In [2]:
import h5py
import numpy as np
import os
import sys
import joblib
import ROOT

sys.path.append("..")
from helpers import filename_for

ROOT.ROOT.EnableThreadSafety()

Welcome to JupyROOT 6.16/00


Read the ROOT file event-by-event. Better not try anything fancy here. Write hdf5 event by event.

In [3]:
def root2hdf5(distance, doubleplane, energy, erel, neutron, physics, subrun):
    inpfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "digi.root")
    outfile = filename_for(distance, doubleplane, energy, erel, neutron, physics, subrun, "trifeature.h5")

    with h5py.File(outfile, "w") as h5file:
        tfile = ROOT.TFile.Open(os.fspath(inpfile))
        ttree = tfile.Get("evt")
        num_events = ttree.GetEntries()
        dim = 6

        data = h5file.create_dataset(
            "data", shape=(num_events, dim), dtype=np.int16, chunks=(1000, dim), compression="gzip", compression_opts=9,
        )

        buff = np.zeros((dim), np.int16)
        i = 0
        for event in ttree:

            # nPN: Number of incoming primary neutrons
            buff[0] = event.NeulandPrimaryTracks.GetEntries()
            # nPP: Number of primary neutrons with an energy deposition in NeuLAND
            buff[1] = event.NeulandPrimaryPoints.GetEntries()
            # nPH: Number of hits that correspond to a energy deposition of a primary neutron
            buff[2] = event.NeulandPrimaryHits.GetEntries()
            # nHits: Number of hits
            buff[3] = event.NeulandHits.GetEntries()
            # nClus: Number of clusters
            buff[4] = event.NeulandClusters.GetEntries()
            # Edep: Total deposited (detected) energy
            buff[5] = round(sum([hit.GetE() for hit in event.NeulandHits]))

            data[i] = buff
            i += 1
        tfile.Close()

In [4]:
joblib.Parallel(n_jobs=-1, backend="multiprocessing")(
    joblib.delayed(root2hdf5)(
        distance=distance,
        doubleplane=doubleplane,
        energy=energy,
        erel=erel,
        neutron=neutron,
        physics=physics,
        subrun=subrun,
    )
    for distance in distances
    for energy in energies
    for doubleplane in doubleplanes
    for neutron in neutrons
    for erel in erels
    for physics in physicss
    for subrun in subruns
);